In [52]:
import os
import mne
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta
from mne.preprocessing import ICA,read_ica

In [53]:
#read the ICA filtered data file from the folder
sample_data_raw_file=('C:/Users/user/Desktop/ICA/06062019 1_ica.fif')
raw = mne.io.read_raw_fif(sample_data_raw_file, preload=True)
raw_temp=raw.copy().crop(529., 539.) #Copy the raw data and crop the 30 sec data basically skype call
raw_ch = raw_temp.pick_channels(['Fp1', 'F7', 'F3', 'FC1', 'FC2', 'F4', 'F8', 'Fp2','O1','O2']) # select onl

Opening raw data file C:/Users/user/Desktop/ICA/06062019 1_ica.fif...
    Range : 0 ... 289919 =      0.000 ...  2264.992 secs
Ready.
Reading 0 ... 289919  =      0.000 ...  2264.992 secs...


<ipython-input-53-1fd7f5f51d58>:3: RuntimeWarning: This filename (C:/Users/user/Desktop/ICA/06062019 1_ica.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  raw = mne.io.read_raw_fif(sample_data_raw_file, preload=True)


In [54]:
Fp1_0 =raw_ch.copy().pick_channels(['Fp1']) 
F7_0 =raw_ch.copy().pick_channels(['F7'])
F3_0 =raw_ch.copy().pick_channels(['F3'])
FC1_0 =raw_ch.copy().pick_channels(['FC1'])
FC2_0 =raw_ch.copy().pick_channels(['FC2'])
F4_0 =raw_ch.copy().pick_channels(['F4'])
F8_0 =raw_ch.copy().pick_channels(['F8'])
Fp2_0 =raw_ch.copy().pick_channels(['Fp2'])

In [55]:
def filter(x,fmin, fmax):
    band=x.filter(fmin, fmax, fir_design='firwin')
    return band

In [56]:
#KTE calculation:
def TKEO(x):
    tkeo = []

    for i in range(0, len(x)):
        if i == 0 or i == len(x) - 1:
            tkeo.append(x[i])
        else:
            tkeo.append((x[i]**2) - (x[i + 1] * x[i - 1]))
        tkeo =np.asarray(tkeo)
        #print('tkeo',tkeo)
        #print((tkeo.shape))
        return tkeo
    

In [57]:

#Signal windowing for tkeo.               #window_hop_length= 0.040 #40ms hop laegth
def Frame(x):
    fs=128
    window_hop_length= 0.040 #40ms hop laegth
    overlap=int(fs*window_hop_length)
    print(overlap,'Overlap')

    window_size=0.025 #20 ms window size,
    framesize=int(window_size*fs) #no of frame
    length=(x.size)       #define length(size) of signal
    print(framesize,'frame size')

    number_of_frames=int(length/overlap)
    print(type(number_of_frames))
    print("number_of_frames=",number_of_frames)

    frames=np.ndarray((number_of_frames,framesize))#np.ndarray
    print(frames[100])
    print('frames len',len(frames))
    print((frames.shape))
    return frames


In [58]:
import numpy as geek
def coffe(x):
    kte_mu=np.mean(x,axis=1) # calculate mean
    
    print((kte_mu))
    std = np.std(x,axis=1)# calculate standard division
    #kte_std=geek.nan_to_num(std) #remove the NAN and inf value from std
    print(std)
    c_kte = kte_mu/(1+np.exp(-std)) #KTE Logistic Coefficients is calculated over every frame.
    print(c_kte)
    return c_kte

In [59]:
# extract the frequency bands using bandpass filter
Th_0 = filter(Fp1_0,4., 8.)
Al_0 =filter(Fp1_0,8., 13.)
Be_0 = filter(Fp1_0,13., 30.)
Ga_0 = filter(Fp1_0,30., 56.)
#get the data from the signal
theta,times = Th_0[:]
alpha,times = Al_0[:]
beta,times = Be_0 [:]
gamma,times = Ga_0[:]

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 8 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 8.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 9.00 Hz)
- Filter length: 211 samples (1.648 sec)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 13 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 13.

In [60]:
Fp1_th0= TKEO(theta)
Fp1_al0= TKEO(alpha)
Fp1_be0= TKEO(beta)
Fp1_ga0= TKEO(gamma)
#############################
f_th0 = Frame(Fp1_th0)
f_al0 = Frame(Fp1_al0)
f_be0 = Frame(Fp1_be0)
f_ga0 = Frame(Fp1_ga0)


3 frame size
number_of_frames= 427
[2.34375   2.3515625 2.359375 ]
frames len 427
(427, 3)
3 frame size
number_of_frames= 427
[9.80128018e-13 3.66443229e-12 5.76111151e-12]
frames len 427
(427, 3)
3 frame size
number_of_frames= 427
[9.80128018e-13 3.66443229e-12 5.76111151e-12]
frames len 427
(427, 3)
3 frame size
number_of_frames= 427
[9.80128018e-13 3.66443229e-12 5.76111151e-12]
frames len 427
(427, 3)


In [61]:
c_kte_th = coffe(f_th0)
c_kte_al = coffe(f_al0)
c_kte_be = coffe(f_be0)
c_kte_ga = coffe(f_ga0)

[7.8125000e-03 3.1250000e-02 5.4687500e-02 7.8125000e-02 1.0156250e-01
 1.2500000e-01 1.4843750e-01 1.7187500e-01 1.9531250e-01 2.1875000e-01
 2.4218750e-01 2.6562500e-01 2.8906250e-01 3.1250000e-01 3.3593750e-01
 3.5937500e-01 3.8281250e-01 4.0625000e-01 4.2968750e-01 4.5312500e-01
 4.7656250e-01 5.0000000e-01 5.2343750e-01 5.4687500e-01 5.7031250e-01
 5.9375000e-01 6.1718750e-01 6.4062500e-01 6.6406250e-01 6.8750000e-01
 7.1093750e-01 7.3437500e-01 7.5781250e-01 7.8125000e-01 8.0468750e-01
 8.2812500e-01 8.5156250e-01 8.7500000e-01 8.9843750e-01 9.2187500e-01
 9.4531250e-01 9.6875000e-01 9.9218750e-01 1.0156250e+00 1.0390625e+00
 1.0625000e+00 1.0859375e+00 1.1093750e+00 1.1328125e+00 1.1562500e+00
 1.1796875e+00 1.2031250e+00 1.2265625e+00 1.2500000e+00 1.2734375e+00
 1.2968750e+00 1.3203125e+00 1.3437500e+00 1.3671875e+00 1.3906250e+00
 1.4140625e+00 1.4375000e+00 1.4609375e+00 1.4843750e+00 1.5078125e+00
 1.5312500e+00 1.5546875e+00 1.5781250e+00 1.6015625e+00 1.6250000e+00
 1.648

[-2.19769054e-12 -5.13212994e-12 -1.67280265e-12  2.46223722e-12
  4.05852635e-12  1.65983965e-12 -2.92044193e-12 -3.94897670e-12
 -9.94097960e-13  3.05632606e-12  4.40064665e-12 -8.87204733e-14
 -4.53854445e-12 -2.45347113e-12  1.68195161e-12  2.47053679e-12
  1.61593476e-12  5.98662787e-14 -2.81356475e-12 -3.16527219e-12
  1.35249699e-12  4.68306181e-12  7.90089149e-13 -3.65507395e-12
 -1.26032465e-12  1.02241196e-12 -4.06838811e-13  1.06636568e-12
  2.64934469e-12 -1.35142348e-12 -3.28987752e-12  5.70948239e-13
  1.56999395e-12 -5.07014477e-13  9.15840043e-13  1.92430534e-12
 -1.67949576e-12 -3.26174550e-12 -4.66800697e-14  2.10799037e-12
  2.18963854e-12  1.78727222e-12 -1.79934657e-12 -5.38344904e-12
 -1.60922681e-12  4.87422412e-12  4.79690772e-12  8.01703041e-14
 -3.86982944e-12 -5.11103230e-12 -1.15094967e-12  5.21337324e-12
  5.72004820e-12 -9.49424334e-14 -5.30843856e-12 -4.79140262e-12
  1.98256186e-13  3.94675903e-12  4.39074645e-12  1.57824729e-12
 -3.63620795e-12 -5.26336

[-1.09884527e-12 -2.56606497e-12 -8.36401326e-13  1.23111861e-12
  2.02926317e-12  8.29919823e-13 -1.46022096e-12 -1.97448835e-12
 -4.97048980e-13  1.52816303e-12  2.20032333e-12 -4.43602367e-14
 -2.26927223e-12 -1.22673557e-12  8.40975803e-13  1.23526839e-12
  8.07967379e-13  2.99331394e-14 -1.40678237e-12 -1.58263609e-12
  6.76248495e-13  2.34153091e-12  3.95044575e-13 -1.82753697e-12
 -6.30162324e-13  5.11205982e-13 -2.03419406e-13  5.33182842e-13
  1.32467234e-12 -6.75711742e-13 -1.64493876e-12  2.85474119e-13
  7.84996976e-13 -2.53507238e-13  4.57920021e-13  9.62152669e-13
 -8.39747881e-13 -1.63087275e-12 -2.33400349e-14  1.05399519e-12
  1.09481927e-12  8.93636112e-13 -8.99673286e-13 -2.69172452e-12
 -8.04613407e-13  2.43711206e-12  2.39845386e-12  4.00851520e-14
 -1.93491472e-12 -2.55551615e-12 -5.75474836e-13  2.60668662e-12
  2.86002410e-12 -4.74712167e-14 -2.65421928e-12 -2.39570131e-12
  9.91280931e-14  1.97337952e-12  2.19537323e-12  7.89123647e-13
 -1.81810398e-12 -2.63168

 -8.41599748e-13 -1.84129774e-12 -7.41954779e-13]
[-2.19769054e-12 -5.13212994e-12 -1.67280265e-12  2.46223722e-12
  4.05852635e-12  1.65983965e-12 -2.92044193e-12 -3.94897670e-12
 -9.94097960e-13  3.05632606e-12  4.40064665e-12 -8.87204733e-14
 -4.53854445e-12 -2.45347113e-12  1.68195161e-12  2.47053679e-12
  1.61593476e-12  5.98662787e-14 -2.81356475e-12 -3.16527219e-12
  1.35249699e-12  4.68306181e-12  7.90089149e-13 -3.65507395e-12
 -1.26032465e-12  1.02241196e-12 -4.06838811e-13  1.06636568e-12
  2.64934469e-12 -1.35142348e-12 -3.28987752e-12  5.70948239e-13
  1.56999395e-12 -5.07014477e-13  9.15840043e-13  1.92430534e-12
 -1.67949576e-12 -3.26174550e-12 -4.66800697e-14  2.10799037e-12
  2.18963854e-12  1.78727222e-12 -1.79934657e-12 -5.38344904e-12
 -1.60922681e-12  4.87422412e-12  4.79690772e-12  8.01703041e-14
 -3.86982944e-12 -5.11103230e-12 -1.15094967e-12  5.21337324e-12
  5.72004820e-12 -9.49424334e-14 -5.30843856e-12 -4.79140262e-12
  1.98256186e-13  3.94675903e-12  4.3907

 8.14158069e-13 1.47963299e-13 1.16584394e-12]
[-1.09884527e-12 -2.56606497e-12 -8.36401326e-13  1.23111861e-12
  2.02926317e-12  8.29919823e-13 -1.46022096e-12 -1.97448835e-12
 -4.97048980e-13  1.52816303e-12  2.20032333e-12 -4.43602367e-14
 -2.26927223e-12 -1.22673557e-12  8.40975803e-13  1.23526839e-12
  8.07967379e-13  2.99331394e-14 -1.40678237e-12 -1.58263609e-12
  6.76248495e-13  2.34153091e-12  3.95044575e-13 -1.82753697e-12
 -6.30162324e-13  5.11205982e-13 -2.03419406e-13  5.33182842e-13
  1.32467234e-12 -6.75711742e-13 -1.64493876e-12  2.85474119e-13
  7.84996976e-13 -2.53507238e-13  4.57920021e-13  9.62152669e-13
 -8.39747881e-13 -1.63087275e-12 -2.33400349e-14  1.05399519e-12
  1.09481927e-12  8.93636112e-13 -8.99673286e-13 -2.69172452e-12
 -8.04613407e-13  2.43711206e-12  2.39845386e-12  4.00851520e-14
 -1.93491472e-12 -2.55551615e-12 -5.75474836e-13  2.60668662e-12
  2.86002410e-12 -4.74712167e-14 -2.65421928e-12 -2.39570131e-12
  9.91280931e-14  1.97337952e-12  2.1953732

In [62]:
import pandas as pd
df_th0 =pd.DataFrame(c_kte_th ,columns = ['FP1/theta'])

df_al0 =pd.DataFrame(c_kte_al ,columns = ['FP1/alpha'])

df_be0 =pd.DataFrame(c_kte_be ,columns = ['FP1/beta'])
df_ga0 =pd.DataFrame(c_kte_ga ,columns = ['FP1/gamma'])
df0_Fp1 = pd.concat([df_th0,df_al0,df_be0,df_ga0],axis =1,
                       sort=False)
#df0_Fp1['phase'] = 1
df0_Fp1


,FP1/theta,FP1/alpha,FP1/beta,FP1/gamma
0,0.003919,-1.098845e-12,-1.098845e-12,-1.098845e-12
1,0.015675,-2.566065e-12,-2.566065e-12,-2.566065e-12
2,0.027431,-8.364013e-13,-8.364013e-13,-8.364013e-13
3,0.039187,1.231119e-12,1.231119e-12,1.231119e-12
4,0.050943,2.029263e-12,2.029263e-12,2.029263e-12
...,...,...,...,...
422,4.965004,1.757150e-12,1.757150e-12,1.757150e-12
423,4.976760,7.371900e-13,7.371900e-13,7.371900e-13
424,4.988516,-8.415997e-13,-8.415997e-13,-8.415997e-13
425,5.000272,-1.841298e-12,-1.841298e-12,-1.841298e-12


In [63]:
# extract the frequency bands using bandpass filter
Th7_0 = filter(F7_0,4., 8.)
Al7_0 =filter(F7_0,8., 13.)
Be7_0 = filter(F7_0,13., 30.)
Ga7_0 = filter(F7_0,30., 56.)
#get the data from the signal
theta7,times = Th7_0[:]
alpha7,times = Al7_0[:]
beta7,times = Be7_0 [:]
gamma7,times = Ga7_0[:]

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 8 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 8.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 9.00 Hz)
- Filter length: 211 samples (1.648 sec)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 13 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 13.

In [64]:
F7_th0= TKEO(theta7)
F7_al0= TKEO(alpha7)
F7_be0= TKEO(beta7)
F7_ga0= TKEO(gamma7)
#############################
f7_th0 = Frame(F7_th0)
f7_al0 = Frame(F7_al0)
f7_be0 = Frame(F7_be0)
f7_ga0 = Frame(F7_ga0)

3 frame size
number_of_frames= 427
[-1.65778491e-12 -7.21876538e-12 -1.12156860e-11]
frames len 427
(427, 3)
3 frame size
number_of_frames= 427
[-1.65778491e-12 -7.21876538e-12 -1.12156860e-11]
frames len 427
(427, 3)
3 frame size
number_of_frames= 427
[-1.65778491e-12 -7.21876538e-12 -1.12156860e-11]
frames len 427
(427, 3)
3 frame size
number_of_frames= 427
[-1.65778491e-12 -7.21876538e-12 -1.12156860e-11]
frames len 427
(427, 3)


In [65]:
c_kte_th7 = coffe(f7_th0)
c_kte_al7 = coffe(f7_al0)
c_kte_be7 = coffe(f7_be0)
c_kte_ga7 = coffe(f7_ga0)

[ 2.70398450e-12  6.77673642e-12  3.30502902e-12 -2.12881524e-12
 -5.25699995e-12 -4.15564679e-12  9.24521141e-13  5.22547104e-12
  5.10498171e-12  9.27551124e-14 -6.46858289e-12 -6.16682943e-12
  1.90510700e-12  6.49613289e-12  3.50938045e-12 -2.68976927e-13
 -3.46282433e-12 -6.57820293e-12 -2.73315170e-12  7.61707283e-12
  8.96572070e-12 -2.46124599e-12 -8.56525894e-12 -3.31494760e-12
  9.76607565e-13  3.68654982e-12  7.33132829e-12  1.97241005e-12
 -8.97230230e-12 -8.21813108e-12  2.98373239e-12  8.97952860e-12
  4.44116914e-12 -3.01590016e-12 -6.15104921e-12 -4.90130469e-12
  6.35697391e-13  8.04315538e-12  6.79893536e-12 -4.10648370e-12
 -9.25472307e-12 -2.72481619e-12  4.54591320e-12  5.94542333e-12
  3.08202014e-12 -2.56052521e-12 -7.01696005e-12 -4.03670339e-12
  4.41300011e-12  7.33329079e-12  1.36826314e-12 -4.13403618e-12
 -4.40770347e-12 -2.60096549e-12  2.10943297e-12  6.65244024e-12
  2.34682255e-12 -4.53418515e-12 -3.58458152e-12 -9.01616668e-13
 -6.81287028e-14  3.77263

[ 1.35199225e-12  3.38836821e-12  1.65251451e-12 -1.06440762e-12
 -2.62849998e-12 -2.07782339e-12  4.62260570e-13  2.61273552e-12
  2.55249086e-12  4.63775562e-14 -3.23429144e-12 -3.08341471e-12
  9.52553499e-13  3.24806644e-12  1.75469022e-12 -1.34488463e-13
 -1.73141216e-12 -3.28910146e-12 -1.36657585e-12  3.80853642e-12
  4.48286035e-12 -1.23062300e-12 -4.28262947e-12 -1.65747380e-12
  4.88303783e-13  1.84327491e-12  3.66566415e-12  9.86205026e-13
 -4.48615115e-12 -4.10906554e-12  1.49186620e-12  4.48976430e-12
  2.22058457e-12 -1.50795008e-12 -3.07552461e-12 -2.45065235e-12
  3.17848696e-13  4.02157769e-12  3.39946768e-12 -2.05324185e-12
 -4.62736153e-12 -1.36240810e-12  2.27295660e-12  2.97271166e-12
  1.54101007e-12 -1.28026261e-12 -3.50848003e-12 -2.01835170e-12
  2.20650005e-12  3.66664540e-12  6.84131571e-13 -2.06701809e-12
 -2.20385174e-12 -1.30048275e-12  1.05471649e-12  3.32622012e-12
  1.17341127e-12 -2.26709258e-12 -1.79229076e-12 -4.50808334e-13
 -3.40643514e-14  1.88631

[ 1.35199225e-12  3.38836821e-12  1.65251451e-12 -1.06440762e-12
 -2.62849998e-12 -2.07782339e-12  4.62260570e-13  2.61273552e-12
  2.55249086e-12  4.63775562e-14 -3.23429144e-12 -3.08341471e-12
  9.52553499e-13  3.24806644e-12  1.75469022e-12 -1.34488463e-13
 -1.73141216e-12 -3.28910146e-12 -1.36657585e-12  3.80853642e-12
  4.48286035e-12 -1.23062300e-12 -4.28262947e-12 -1.65747380e-12
  4.88303783e-13  1.84327491e-12  3.66566415e-12  9.86205026e-13
 -4.48615115e-12 -4.10906554e-12  1.49186620e-12  4.48976430e-12
  2.22058457e-12 -1.50795008e-12 -3.07552461e-12 -2.45065235e-12
  3.17848696e-13  4.02157769e-12  3.39946768e-12 -2.05324185e-12
 -4.62736153e-12 -1.36240810e-12  2.27295660e-12  2.97271166e-12
  1.54101007e-12 -1.28026261e-12 -3.50848003e-12 -2.01835170e-12
  2.20650005e-12  3.66664540e-12  6.84131571e-13 -2.06701809e-12
 -2.20385174e-12 -1.30048275e-12  1.05471649e-12  3.32622012e-12
  1.17341127e-12 -2.26709258e-12 -1.79229076e-12 -4.50808334e-13
 -3.40643514e-14  1.88631

[2.11405329e-12 2.43271120e-13 1.59847041e-12 1.43579427e-12
 2.21844046e-13 1.06039181e-12 1.42182848e-12 6.74776831e-13
 6.55598384e-13 1.90567252e-12 1.37275339e-12 1.51945894e-12
 2.42555204e-12 2.02615235e-13 1.37697589e-12 7.79824262e-13
 8.86730249e-13 4.94987526e-13 2.37614039e-12 2.62368600e-12
 1.98387725e-12 3.51701699e-12 5.42616504e-13 1.85728147e-12
 5.14301222e-13 1.20841445e-12 3.64761483e-13 3.06071143e-12
 2.03095251e-12 2.29247057e-12 3.07307497e-12 3.23894927e-13
 2.18358736e-12 1.62117289e-12 1.88874378e-13 8.56535571e-13
 2.15241645e-12 1.44185421e-12 2.24054426e-12 3.00483826e-12
 5.94256038e-13 2.50467707e-12 1.29922445e-12 3.76366718e-13
 1.25642885e-12 1.67497528e-12 5.04526451e-13 2.04151907e-12
 2.11533448e-12 8.09886364e-13 2.04842775e-12 6.85979994e-13
 1.99012058e-13 8.34360973e-13 1.80753955e-12 3.24022164e-13
 2.26894209e-12 9.07779097e-13 1.01329591e-12 2.32465189e-13
 6.02400774e-13 1.29459833e-12 1.04727822e-12 2.11390696e-12
 3.36120901e-13 1.445322

In [66]:
df7_th0 =pd.DataFrame(c_kte_th7 ,columns = ['F7/theta'])

df7_al0 =pd.DataFrame(c_kte_al7 ,columns = ['F7/alpha'])

df7_be0 =pd.DataFrame(c_kte_be7 ,columns = ['F7/beta'])
df7_ga0 =pd.DataFrame(c_kte_ga7 ,columns = ['F7/gamma'])
df7_F7 = pd.concat([df7_th0,df7_al0,df7_be0,df7_ga0],axis =1,sort=False)

#df7_F7['phase'] = 1                       
df7_F7

,F7/theta,F7/alpha,F7/beta,F7/gamma
0,1.351992e-12,1.351992e-12,1.351992e-12,1.351992e-12
1,3.388368e-12,3.388368e-12,3.388368e-12,3.388368e-12
2,1.652515e-12,1.652515e-12,1.652515e-12,1.652515e-12
3,-1.064408e-12,-1.064408e-12,-1.064408e-12,-1.064408e-12
4,-2.628500e-12,-2.628500e-12,-2.628500e-12,-2.628500e-12
...,...,...,...,...
422,-4.426104e-13,-4.426104e-13,-4.426104e-13,-4.426104e-13
423,-1.434249e-12,-1.434249e-12,-1.434249e-12,-1.434249e-12
424,-4.500093e-13,-4.500093e-13,-4.500093e-13,-4.500093e-13
425,1.075590e-12,1.075590e-12,1.075590e-12,1.075590e-12


In [67]:
# extract the frequency bands using bandpass filter
Th3_0 = filter(F3_0,4., 8.)
Al3_0 =filter(F3_0,8., 13.)
Be3_0 = filter(F3_0,13., 30.)
Ga3_0 = filter(F3_0,30., 56.)
#get the data from the signal
theta3,times = Th3_0[:]
alpha3,times = Al3_0[:]
beta3,times = Be3_0 [:]
gamma3,times = Ga3_0[:]

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 8 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 8.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 9.00 Hz)
- Filter length: 211 samples (1.648 sec)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 13 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 13.

In [68]:
F3_th0= TKEO(theta3)
F3_al0= TKEO(alpha3)
F3_be0= TKEO(beta3)
F3_ga0= TKEO(gamma3)
#############################
f3_th0 = Frame(F3_th0)
f3_al0 = Frame(F3_al0)
f3_be0 = Frame(F3_be0)
f3_ga0 = Frame(F3_ga0)

3 frame size
number_of_frames= 427
[-3.24344321e-13 -2.46665746e-12 -3.61057051e-12]
frames len 427
(427, 3)
3 frame size
number_of_frames= 427
[-3.24344321e-13 -2.46665746e-12 -3.61057051e-12]
frames len 427
(427, 3)
3 frame size
number_of_frames= 427
[-3.24344321e-13 -2.46665746e-12 -3.61057051e-12]
frames len 427
(427, 3)
3 frame size
number_of_frames= 427
[-3.24344321e-13 -2.46665746e-12 -3.61057051e-12]
frames len 427
(427, 3)


In [69]:
c_kte_th3 = coffe(f3_th0)
c_kte_al3 = coffe(f3_al0)
c_kte_be3 = coffe(f3_be0)
c_kte_ga3 = coffe(f3_ga0)

[-3.64058080e-12 -1.17836867e-11 -1.08467254e-11  2.72850157e-12
  1.56510458e-11  1.01335901e-11 -6.92917136e-12 -1.28672887e-11
 -4.59508789e-12  2.78279824e-12  6.74698322e-12  8.92016483e-12
  1.05952105e-12 -1.04160061e-11 -8.50005014e-12  2.29484962e-12
  6.46222202e-12  4.69417543e-12  2.71235791e-12 -2.66481144e-12
 -8.62259973e-12 -5.10857582e-12  5.52104170e-12  8.56808599e-12
  2.00000457e-12 -2.75288501e-12 -5.18108254e-12 -6.74265286e-12
  7.32461962e-13  1.06866070e-11  5.97894174e-12 -6.29332848e-12
 -7.16800090e-12 -9.69502674e-13  9.89406955e-13  4.08257757e-12
  7.14260861e-12 -1.49798936e-12 -1.03458996e-11 -2.46718848e-12
  6.69379481e-12  3.05940076e-12 -1.84133712e-13  2.69724626e-13
 -3.71765641e-12 -4.52658245e-12  3.13315602e-12  6.36417359e-12
 -4.00234039e-13 -4.77135848e-12 -1.18227931e-12  1.41745719e-12
  3.65504968e-13  1.40589001e-12  1.95802374e-12 -2.80243506e-12
 -3.96693751e-12  2.97894979e-12  4.58841206e-12 -2.24654170e-12
 -4.20283055e-12  8.98459

[-3.64058080e-12 -1.17836867e-11 -1.08467254e-11  2.72850157e-12
  1.56510458e-11  1.01335901e-11 -6.92917136e-12 -1.28672887e-11
 -4.59508789e-12  2.78279824e-12  6.74698322e-12  8.92016483e-12
  1.05952105e-12 -1.04160061e-11 -8.50005014e-12  2.29484962e-12
  6.46222202e-12  4.69417543e-12  2.71235791e-12 -2.66481144e-12
 -8.62259973e-12 -5.10857582e-12  5.52104170e-12  8.56808599e-12
  2.00000457e-12 -2.75288501e-12 -5.18108254e-12 -6.74265286e-12
  7.32461962e-13  1.06866070e-11  5.97894174e-12 -6.29332848e-12
 -7.16800090e-12 -9.69502674e-13  9.89406955e-13  4.08257757e-12
  7.14260861e-12 -1.49798936e-12 -1.03458996e-11 -2.46718848e-12
  6.69379481e-12  3.05940076e-12 -1.84133712e-13  2.69724626e-13
 -3.71765641e-12 -4.52658245e-12  3.13315602e-12  6.36417359e-12
 -4.00234039e-13 -4.77135848e-12 -1.18227931e-12  1.41745719e-12
  3.65504968e-13  1.40589001e-12  1.95802374e-12 -2.80243506e-12
 -3.96693751e-12  2.97894979e-12  4.58841206e-12 -2.24654170e-12
 -4.20283055e-12  8.98459

 -2.54390043e-12 -2.80696905e-12 -4.03090396e-13]
[2.87002511e-12 1.67592737e-12 2.29363274e-12 4.87638138e-12
 1.53619614e-12 4.53763543e-12 3.86232083e-12 1.38032587e-12
 2.36951682e-12 1.24073944e-12 1.29741459e-12 4.38234653e-13
 3.71345951e-12 2.02722502e-12 2.91088256e-12 2.48089005e-12
 3.55235544e-13 4.76502440e-13 6.39556318e-13 2.03147547e-12
 1.07099760e-12 2.73824472e-12 2.72886199e-12 1.20053827e-12
 1.98478539e-12 5.96714652e-13 9.05391243e-13 5.73551230e-13
 3.41332631e-12 1.22791171e-12 3.43375848e-12 2.28598873e-12
 1.46194707e-12 1.27819919e-12 3.26552436e-13 1.42014454e-12
 7.22464966e-13 3.59468009e-12 6.69838903e-13 3.61057179e-12
 8.48529355e-13 1.82822278e-12 1.32434307e-13 2.77717450e-13
 1.57281183e-12 1.31720149e-12 2.49678279e-12 1.18183215e-12
 2.16151662e-12 5.73384320e-13 9.85673400e-13 3.23496147e-13
 2.75385054e-13 4.62152733e-13 9.93634180e-13 1.07845678e-12
 1.07075053e-12 1.83248720e-12 1.04736967e-12 2.07637434e-12
 1.11290296e-12 1.41108007e-12 6.86

[2.87002511e-12 1.67592737e-12 2.29363274e-12 4.87638138e-12
 1.53619614e-12 4.53763543e-12 3.86232083e-12 1.38032587e-12
 2.36951682e-12 1.24073944e-12 1.29741459e-12 4.38234653e-13
 3.71345951e-12 2.02722502e-12 2.91088256e-12 2.48089005e-12
 3.55235544e-13 4.76502440e-13 6.39556318e-13 2.03147547e-12
 1.07099760e-12 2.73824472e-12 2.72886199e-12 1.20053827e-12
 1.98478539e-12 5.96714652e-13 9.05391243e-13 5.73551230e-13
 3.41332631e-12 1.22791171e-12 3.43375848e-12 2.28598873e-12
 1.46194707e-12 1.27819919e-12 3.26552436e-13 1.42014454e-12
 7.22464966e-13 3.59468009e-12 6.69838903e-13 3.61057179e-12
 8.48529355e-13 1.82822278e-12 1.32434307e-13 2.77717450e-13
 1.57281183e-12 1.31720149e-12 2.49678279e-12 1.18183215e-12
 2.16151662e-12 5.73384320e-13 9.85673400e-13 3.23496147e-13
 2.75385054e-13 4.62152733e-13 9.93634180e-13 1.07845678e-12
 1.07075053e-12 1.83248720e-12 1.04736967e-12 2.07637434e-12
 1.11290296e-12 1.41108007e-12 6.86126310e-13 4.56972974e-13
 1.90324863e-13 2.789851

 -1.27195021e-12 -1.40348452e-12 -2.01545198e-13]


In [70]:
df3_th0 =pd.DataFrame(c_kte_th3 ,columns = ['F3/theta'])

df3_al0 =pd.DataFrame(c_kte_al3 ,columns = ['F3/alpha'])

df3_be0 =pd.DataFrame(c_kte_be3 ,columns = ['F3/beta'])
df3_ga0 =pd.DataFrame(c_kte_ga3 ,columns = ['F3/gamma'])
df3_F3 = pd.concat([df3_th0,df3_al0,df3_be0,df3_ga0],axis =1,sort=False)

#df3_F3['phase'] = 1                       
df3_F3

,F3/theta,F3/alpha,F3/beta,F3/gamma
0,-1.820290e-12,-1.820290e-12,-1.820290e-12,-1.820290e-12
1,-5.891843e-12,-5.891843e-12,-5.891843e-12,-5.891843e-12
2,-5.423363e-12,-5.423363e-12,-5.423363e-12,-5.423363e-12
3,1.364251e-12,1.364251e-12,1.364251e-12,1.364251e-12
4,7.825523e-12,7.825523e-12,7.825523e-12,7.825523e-12
...,...,...,...,...
422,1.681998e-12,1.681998e-12,1.681998e-12,1.681998e-12
423,1.618420e-12,1.618420e-12,1.618420e-12,1.618420e-12
424,-1.271950e-12,-1.271950e-12,-1.271950e-12,-1.271950e-12
425,-1.403485e-12,-1.403485e-12,-1.403485e-12,-1.403485e-12


In [71]:
# extract the frequency bands using bandpass filter
Th_c1_0 = filter(FC1_0,4., 8.)
Al_c1_0 =filter(FC1_0,8., 13.)
Be_c1_0 = filter(FC1_0,13., 30.)
Ga_c1_0 = filter(FC1_0,30., 56.)
#get the data from the signal
theta_c1,times = Th_c1_0[:]
alpha_c1,times = Al_c1_0[:]
beta_c1,times = Be_c1_0 [:]
gamma_c1,times = Ga_c1_0[:]

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 8 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 8.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 9.00 Hz)
- Filter length: 211 samples (1.648 sec)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 13 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 13.

In [72]:
FC1_th0= TKEO(theta_c1)
FC1_al0= TKEO(alpha_c1)
FC1_be0= TKEO(beta_c1)
FC1_ga0= TKEO(gamma_c1)
#############################
fc1_th0 = Frame(FC1_th0)
fc1_al0 = Frame(FC1_al0)
fc1_be0 = Frame(FC1_be0)
fc1_ga0 = Frame(FC1_ga0)

3 frame size
number_of_frames= 427
[-5.45288164e-13  1.57443369e-12  3.36408002e-12]
frames len 427
(427, 3)
3 frame size
number_of_frames= 427
[-5.45288164e-13  1.57443369e-12  3.36408002e-12]
frames len 427
(427, 3)
3 frame size
number_of_frames= 427
[-5.45288164e-13  1.57443369e-12  3.36408002e-12]
frames len 427
(427, 3)
3 frame size
number_of_frames= 427
[-5.45288164e-13  1.57443369e-12  3.36408002e-12]
frames len 427
(427, 3)


In [73]:
c_kte_th_c1 = coffe(fc1_th0)
c_kte_al_c1 = coffe(fc1_al0)
c_kte_be_c1 = coffe(fc1_be0)
c_kte_ga_c1 = coffe(fc1_ga0)

[ 1.90467983e-12  4.45915338e-12  2.01738143e-12 -5.26075949e-13
 -2.48173420e-12 -4.09089256e-12 -1.85167794e-12  4.26265830e-12
  6.51603414e-12  4.23931881e-13 -5.72308119e-12 -4.99303783e-12
 -9.32332659e-13  4.08974964e-12  7.36917435e-12  2.19992534e-12
 -7.38056624e-12 -8.24420979e-12  1.11286688e-12  8.23892726e-12
  5.04612049e-12 -1.86372417e-12 -5.14215531e-12 -5.22020599e-12
 -1.38150024e-13  7.14252407e-12  4.97530679e-12 -3.65728922e-12
 -4.98216304e-12 -3.18286636e-13  1.35892874e-12  5.72011499e-13
  1.11024345e-12  1.74205078e-12 -8.26160430e-14 -2.24925947e-12
 -1.59187619e-12 -6.34954612e-13 -1.34703641e-13  3.41902688e-12
  5.07481169e-12 -1.74015544e-12 -8.20320063e-12 -3.69024481e-12
  5.06262008e-12  6.97763085e-12  2.56659176e-12 -3.61108235e-12
 -7.75631501e-12 -4.12755961e-12  5.53032971e-12  9.00463204e-12
  1.70195689e-12 -6.18112978e-12 -6.53843540e-12 -1.58688085e-12
  4.45128396e-12  6.83483122e-12  1.74945810e-12 -5.23670150e-12
 -5.27256180e-12 -2.32165

[1.49887807e-12 2.35245152e-13 8.95112872e-13 5.23830998e-13
 5.78631338e-13 1.57462860e-13 1.35500069e-12 1.63543460e-12
 6.92860609e-13 2.29800539e-12 7.54138189e-13 9.19127306e-13
 1.16384733e-12 1.52399398e-12 2.42946299e-13 2.70087422e-12
 1.96296763e-12 1.58317453e-12 2.99031893e-12 6.53010349e-13
 2.00239226e-12 1.48255896e-12 3.55884071e-13 4.57774614e-13
 2.18588591e-12 1.30804609e-12 2.26958591e-12 1.87163240e-12
 1.04447266e-12 1.09423019e-12 1.06593299e-13 1.61464117e-13
 2.94469162e-13 9.37125873e-14 8.21183875e-13 2.71323768e-13
 4.89692236e-13 2.76591714e-14 4.66847903e-13 1.25789927e-12
 7.52179962e-13 2.61098950e-12 5.39809821e-13 2.49834735e-12
 1.77338312e-12 6.04423379e-13 1.63816566e-12 1.63683257e-12
 4.26261640e-13 2.26174655e-12 2.52573593e-12 8.76378099e-13
 2.75919549e-12 1.19635944e-12 8.59172349e-13 1.68226228e-12
 1.43735834e-12 3.95629630e-13 2.17715345e-12 1.29417600e-12
 1.11845941e-12 1.34997709e-12 2.61268527e-13 9.36236102e-14
 3.43952959e-13 1.561124

  6.69617625e-13 -4.39282894e-13 -3.06497489e-13]
[1.49887807e-12 2.35245152e-13 8.95112872e-13 5.23830998e-13
 5.78631338e-13 1.57462860e-13 1.35500069e-12 1.63543460e-12
 6.92860609e-13 2.29800539e-12 7.54138189e-13 9.19127306e-13
 1.16384733e-12 1.52399398e-12 2.42946299e-13 2.70087422e-12
 1.96296763e-12 1.58317453e-12 2.99031893e-12 6.53010349e-13
 2.00239226e-12 1.48255896e-12 3.55884071e-13 4.57774614e-13
 2.18588591e-12 1.30804609e-12 2.26958591e-12 1.87163240e-12
 1.04447266e-12 1.09423019e-12 1.06593299e-13 1.61464117e-13
 2.94469162e-13 9.37125873e-14 8.21183875e-13 2.71323768e-13
 4.89692236e-13 2.76591714e-14 4.66847903e-13 1.25789927e-12
 7.52179962e-13 2.61098950e-12 5.39809821e-13 2.49834735e-12
 1.77338312e-12 6.04423379e-13 1.63816566e-12 1.63683257e-12
 4.26261640e-13 2.26174655e-12 2.52573593e-12 8.76378099e-13
 2.75919549e-12 1.19635944e-12 8.59172349e-13 1.68226228e-12
 1.43735834e-12 3.95629630e-13 2.17715345e-12 1.29417600e-12
 1.11845941e-12 1.34997709e-12 2.61

In [74]:
dfc1_th0 =pd.DataFrame(c_kte_th_c1 ,columns = ['FC1/theta'])

dfc1_al0 =pd.DataFrame(c_kte_al_c1 ,columns = ['FC1/alpha'])

dfc1_be0 =pd.DataFrame(c_kte_be_c1 ,columns = ['FC1/beta'])
dfc1_ga0 =pd.DataFrame(c_kte_ga_c1 ,columns = ['FC1/gamma'])
dfc1_FC1 = pd.concat([dfc1_th0,dfc1_al0,dfc1_be0,dfc1_ga0],axis =1,sort=False)

#dfc1_FC1['phase'] = 1                       
dfc1_FC1

,FC1/theta,FC1/alpha,FC1/beta,FC1/gamma
0,9.523399e-13,9.523399e-13,9.523399e-13,9.523399e-13
1,2.229577e-12,2.229577e-12,2.229577e-12,2.229577e-12
2,1.008691e-12,1.008691e-12,1.008691e-12,1.008691e-12
3,-2.630380e-13,-2.630380e-13,-2.630380e-13,-2.630380e-13
4,-1.240867e-12,-1.240867e-12,-1.240867e-12,-1.240867e-12
...,...,...,...,...
422,-3.644093e-13,-3.644093e-13,-3.644093e-13,-3.644093e-13
423,2.986285e-13,2.986285e-13,2.986285e-13,2.986285e-13
424,3.348088e-13,3.348088e-13,3.348088e-13,3.348088e-13
425,-2.196414e-13,-2.196414e-13,-2.196414e-13,-2.196414e-13


In [75]:
# extract the frequency bands using bandpass filter
Th_c2_0 = filter(FC2_0,4., 8.)
Al_c2_0 =filter(FC2_0,8., 13.)
Be_c2_0 = filter(FC2_0,13., 30.)
Ga_c2_0 = filter(FC2_0,30., 56.)
#get the data from the signal
theta_c2,times = Th_c2_0[:]
alpha_c2,times = Al_c2_0[:]
beta_c2,times = Be_c2_0 [:]
gamma_c2,times = Ga_c2_0[:]

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 8 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 8.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 9.00 Hz)
- Filter length: 211 samples (1.648 sec)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 13 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 13.

In [76]:
FC2_th0= TKEO(theta_c2)
FC2_al0= TKEO(alpha_c2)
FC2_be0= TKEO(beta_c2)
FC2_ga0= TKEO(gamma_c2)
#############################
fc2_th0 = Frame(FC2_th0)
fc2_al0 = Frame(FC2_al0)
fc2_be0 = Frame(FC2_be0)
fc2_ga0 = Frame(FC2_ga0)

3 frame size
number_of_frames= 427
[-3.80092722e-12 -2.43032684e-12 -3.79846863e-13]
frames len 427
(427, 3)
3 frame size
number_of_frames= 427
[-3.80092722e-12 -2.43032684e-12 -3.79846863e-13]
frames len 427
(427, 3)
3 frame size
number_of_frames= 427
[-3.80092722e-12 -2.43032684e-12 -3.79846863e-13]
frames len 427
(427, 3)
3 frame size
number_of_frames= 427
[-3.80092722e-12 -2.43032684e-12 -3.79846863e-13]
frames len 427
(427, 3)


In [77]:
c_kte_th_c2 = coffe(fc2_th0)
c_kte_al_c2 = coffe(fc2_al0)
c_kte_be_c2 = coffe(fc2_be0)
c_kte_ga_c2 = coffe(fc2_ga0)

[-3.00625371e-12 -7.38998484e-12 -2.20109803e-12  4.26421656e-12
  4.99998473e-12  2.20585255e-12 -3.15974967e-12 -7.32172157e-12
 -2.23130145e-12  6.69714972e-12  7.09031100e-12 -3.25236584e-13
 -6.91306597e-12 -6.52304324e-12  8.08440998e-13  7.78458402e-12
  6.74774077e-12 -2.42100627e-12 -9.20456888e-12 -4.59829735e-12
  3.97491010e-12  6.79159651e-12  4.39587181e-12 -2.30572805e-12
 -8.32630708e-12 -4.26484622e-12  4.88372522e-12  6.64742816e-12
  1.71250918e-12 -2.79690928e-12 -4.83779495e-12 -3.43462521e-12
  2.19988606e-12  6.18104593e-12  2.28364085e-12 -3.65964982e-12
 -3.47674783e-12 -7.07904145e-13  4.83621632e-13  2.68381568e-12
  3.28489803e-12 -1.35640098e-12 -4.07312909e-12 -6.50760335e-13
  1.61644031e-12  8.49939077e-13  1.19293264e-12  3.34932009e-13
 -2.01653022e-12 -1.14218720e-12  1.04721650e-12  9.44118497e-13
  2.27180008e-14 -3.92813682e-13 -1.39178559e-13  4.88295602e-14
 -9.83532593e-14  4.41802897e-13 -9.96509878e-14 -1.30403781e-12
  3.43178002e-13  1.60047

[2.38080395e-12 3.84987309e-13 2.18282211e-12 1.06586230e-12
 4.52431391e-13 1.09440670e-12 1.76301140e-12 3.05647684e-13
 2.63421239e-12 1.63439938e-12 1.30911998e-12 2.34005605e-12
 1.12251831e-12 1.34572283e-12 2.44424914e-12 1.03465836e-12
 1.65673591e-12 2.88324313e-12 5.49879538e-13 2.55194149e-12
 1.74968076e-12 1.80765342e-13 1.20583795e-12 2.25886106e-12
 6.59064056e-13 2.56769807e-12 1.88579801e-12 8.95955647e-13
 1.47773161e-12 8.99068382e-13 2.64617409e-13 1.09273488e-12
 1.80307529e-12 2.55697395e-13 1.92935341e-12 9.15476903e-13
 7.92798702e-13 4.84774487e-13 3.80872662e-13 7.36050174e-13
 6.85361435e-13 1.53154106e-12 3.51817752e-13 1.19923845e-12
 1.08527985e-13 1.62742520e-13 1.72080038e-13 6.54896627e-13
 3.88978246e-13 6.99217371e-13 3.61471353e-13 2.85129965e-13
 2.16273733e-13 2.01187629e-14 1.11829390e-13 6.45594385e-14
 8.77997339e-14 1.36202067e-13 4.76903627e-13 1.02008353e-13
 7.45950384e-13 2.41518925e-13 4.65857817e-13 2.07380346e-13
 2.39359513e-13 1.758009

[2.38080395e-12 3.84987309e-13 2.18282211e-12 1.06586230e-12
 4.52431391e-13 1.09440670e-12 1.76301140e-12 3.05647684e-13
 2.63421239e-12 1.63439938e-12 1.30911998e-12 2.34005605e-12
 1.12251831e-12 1.34572283e-12 2.44424914e-12 1.03465836e-12
 1.65673591e-12 2.88324313e-12 5.49879538e-13 2.55194149e-12
 1.74968076e-12 1.80765342e-13 1.20583795e-12 2.25886106e-12
 6.59064056e-13 2.56769807e-12 1.88579801e-12 8.95955647e-13
 1.47773161e-12 8.99068382e-13 2.64617409e-13 1.09273488e-12
 1.80307529e-12 2.55697395e-13 1.92935341e-12 9.15476903e-13
 7.92798702e-13 4.84774487e-13 3.80872662e-13 7.36050174e-13
 6.85361435e-13 1.53154106e-12 3.51817752e-13 1.19923845e-12
 1.08527985e-13 1.62742520e-13 1.72080038e-13 6.54896627e-13
 3.88978246e-13 6.99217371e-13 3.61471353e-13 2.85129965e-13
 2.16273733e-13 2.01187629e-14 1.11829390e-13 6.45594385e-14
 8.77997339e-14 1.36202067e-13 4.76903627e-13 1.02008353e-13
 7.45950384e-13 2.41518925e-13 4.65857817e-13 2.07380346e-13
 2.39359513e-13 1.758009

 -4.53325630e-13  5.56946729e-13  3.19474580e-13]


In [78]:
dfc2_th0 =pd.DataFrame(c_kte_th_c2 ,columns = ['FC2/theta'])

dfc2_al0 =pd.DataFrame(c_kte_al_c2 ,columns = ['FC2/alpha'])

dfc2_be0 =pd.DataFrame(c_kte_be_c2 ,columns = ['FC2/beta'])
dfc2_ga0 =pd.DataFrame(c_kte_ga_c2 ,columns = ['FC2/gamma'])
dfc2_FC2 = pd.concat([dfc2_th0,dfc2_al0,dfc2_be0,dfc2_ga0],axis =1,sort=False)

#dfc2_FC2['phase'] = 1                      
dfc2_FC2

,FC2/theta,FC2/alpha,FC2/beta,FC2/gamma
0,-1.503127e-12,-1.503127e-12,-1.503127e-12,-1.503127e-12
1,-3.694992e-12,-3.694992e-12,-3.694992e-12,-3.694992e-12
2,-1.100549e-12,-1.100549e-12,-1.100549e-12,-1.100549e-12
3,2.132108e-12,2.132108e-12,2.132108e-12,2.132108e-12
4,2.499992e-12,2.499992e-12,2.499992e-12,2.499992e-12
...,...,...,...,...
422,-2.633749e-13,-2.633749e-13,-2.633749e-13,-2.633749e-13
423,-1.128669e-12,-1.128669e-12,-1.128669e-12,-1.128669e-12
424,-4.533256e-13,-4.533256e-13,-4.533256e-13,-4.533256e-13
425,5.569467e-13,5.569467e-13,5.569467e-13,5.569467e-13


In [79]:
# extract the frequency bands using bandpass filter
Th_f4_0 = filter(F4_0,4., 8.)
Al_f4_0 =filter(F4_0,8., 13.)
Be_f4_0 = filter(F4_0,13., 30.)
Ga_f4_0 = filter(F4_0,30., 56.)
#get the data from the signal
theta_f4,times = Th_f4_0[:]
alpha_f4,times = Al_f4_0[:]
beta_f4,times = Be_f4_0 [:]
gamma_f4,times = Ga_f4_0[:]

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 8 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 8.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 9.00 Hz)
- Filter length: 211 samples (1.648 sec)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 13 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 13.

In [80]:
F4_th0= TKEO(theta_f4)
F4_al0= TKEO(alpha_f4)
F4_be0= TKEO(beta_f4)
F4_ga0= TKEO(gamma_f4)
#############################
f4_th0 = Frame(F4_th0)
f4_al0 = Frame(F4_al0)
f4_be0 = Frame(F4_be0)
f4_ga0 = Frame(F4_ga0)

3 frame size
number_of_frames= 427
[4.03128674e-12 2.94993709e-12 9.39764783e-13]
frames len 427
(427, 3)
3 frame size
number_of_frames= 427
[0. 0. 0.]
frames len 427
(427, 3)
3 frame size
number_of_frames= 427
[4.03128674e-12 2.94993709e-12 9.39764783e-13]
frames len 427
(427, 3)
3 frame size
number_of_frames= 427
[4.03128674e-12 2.94993709e-12 9.39764783e-13]
frames len 427
(427, 3)


In [81]:
c_kte_th_f4 = coffe(f4_th0)
c_kte_al_f4 = coffe(f4_al0)
c_kte_be_f4 = coffe(f4_be0)
c_kte_ga_f4 = coffe(f4_ga0)

[ 3.27801147e-12  9.05642768e-12  5.54959290e-12 -3.74055320e-12
 -1.03610666e-11 -4.74464505e-12  6.64579149e-12  8.92460878e-12
  3.01780220e-12 -5.02423918e-12 -1.02814145e-11 -3.69436804e-12
  8.26779193e-12  9.56552465e-12  1.29418720e-12 -6.98555229e-12
 -9.27347969e-12 -2.47844775e-12  7.64768441e-12  1.05099701e-11
  2.48259404e-12 -9.34645981e-12 -1.10938268e-11 -4.25477925e-13
  8.78222127e-12  9.88855451e-12  3.57501331e-12 -9.54949847e-12
 -1.49544799e-11 -1.26395098e-12  1.41976613e-11  1.25836306e-11
 -9.17174075e-13 -1.30075944e-11 -1.34834654e-11  7.42110807e-13
  1.61033010e-11  1.39616579e-11 -4.47687498e-12 -1.75273872e-11
 -1.09475635e-11  6.10178033e-12  1.63170969e-11  1.03482395e-11
 -6.96644802e-12 -1.73355076e-11 -8.46275636e-12  9.11445825e-12
  1.59913733e-11  6.14938053e-12 -9.43449662e-12 -1.43302110e-11
 -4.16528236e-12  8.96869529e-12  1.18957685e-11  3.38590413e-12
 -7.34227745e-12 -1.05667344e-11 -3.68334026e-12  6.85074491e-12
  9.78403869e-12  2.07660

 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

[2.63889551e-12 3.35406751e-13 1.87451288e-12 2.75521273e-12
 6.64719787e-13 3.36827170e-12 2.28738583e-12 1.03587055e-12
 1.90412866e-12 2.38090161e-12 3.48527241e-13 3.66110310e-12
 2.17402572e-12 1.63612928e-12 2.33211293e-12 1.85071315e-12
 4.70508252e-13 3.13568291e-12 2.16274854e-12 9.79530661e-13
 3.26999527e-12 2.48324547e-12 1.70982141e-12 3.44300702e-12
 1.37136133e-12 6.72861039e-13 2.87501870e-12 3.67692436e-12
 1.39363314e-12 5.45530038e-12 2.23395841e-12 2.82334114e-12
 4.04046236e-12 2.13643593e-12 2.05711636e-12 5.13326901e-12
 2.56870873e-12 3.70737852e-12 5.48272390e-12 1.15196108e-12
 4.08182782e-12 4.56142604e-12 8.17624184e-13 3.89371525e-12
 4.74303885e-12 6.94694326e-13 4.58927250e-12 4.20339819e-12
 7.67918760e-13 4.33683906e-12 3.49631662e-12 1.00630881e-12
 4.17263232e-12 2.60166799e-12 1.30857789e-12 3.02512539e-12
 2.20987066e-12 4.96462634e-13 2.99610874e-12 2.44064869e-12
 1.08495270e-12 2.72329448e-12 1.22941459e-12 6.47495389e-13
 1.33140238e-12 1.279825

[ 3.27801147e-12  9.05642768e-12  5.54959290e-12 -3.74055320e-12
 -1.03610666e-11 -4.74464505e-12  6.64579149e-12  8.92460878e-12
  3.01780220e-12 -5.02423918e-12 -1.02814145e-11 -3.69436804e-12
  8.26779193e-12  9.56552465e-12  1.29418720e-12 -6.98555229e-12
 -9.27347969e-12 -2.47844775e-12  7.64768441e-12  1.05099701e-11
  2.48259404e-12 -9.34645981e-12 -1.10938268e-11 -4.25477925e-13
  8.78222127e-12  9.88855451e-12  3.57501331e-12 -9.54949847e-12
 -1.49544799e-11 -1.26395098e-12  1.41976613e-11  1.25836306e-11
 -9.17174075e-13 -1.30075944e-11 -1.34834654e-11  7.42110807e-13
  1.61033010e-11  1.39616579e-11 -4.47687498e-12 -1.75273872e-11
 -1.09475635e-11  6.10178033e-12  1.63170969e-11  1.03482395e-11
 -6.96644802e-12 -1.73355076e-11 -8.46275636e-12  9.11445825e-12
  1.59913733e-11  6.14938053e-12 -9.43449662e-12 -1.43302110e-11
 -4.16528236e-12  8.96869529e-12  1.18957685e-11  3.38590413e-12
 -7.34227745e-12 -1.05667344e-11 -3.68334026e-12  6.85074491e-12
  9.78403869e-12  2.07660

 9.30848987e-13 1.07378537e-12 1.06917396e-12]
[ 1.63900573e-12  4.52821384e-12  2.77479645e-12 -1.87027660e-12
 -5.18053330e-12 -2.37232252e-12  3.32289574e-12  4.46230439e-12
  1.50890110e-12 -2.51211959e-12 -5.14070725e-12 -1.84718402e-12
  4.13389596e-12  4.78276233e-12  6.47093600e-13 -3.49277614e-12
 -4.63673984e-12 -1.23922388e-12  3.82384220e-12  5.25498506e-12
  1.24129702e-12 -4.67322991e-12 -5.54691339e-12 -2.12738962e-13
  4.39111064e-12  4.94427726e-12  1.78750665e-12 -4.77474923e-12
 -7.47723993e-12 -6.31975491e-13  7.09883067e-12  6.29181531e-12
 -4.58587038e-13 -6.50379718e-12 -6.74173268e-12  3.71055403e-13
  8.05165050e-12  6.98082896e-12 -2.23843749e-12 -8.76369360e-12
 -5.47378177e-12  3.05089017e-12  8.15854845e-12  5.17411975e-12
 -3.48322401e-12 -8.66775382e-12 -4.23137818e-12  4.55722912e-12
  7.99568664e-12  3.07469026e-12 -4.71724831e-12 -7.16510552e-12
 -2.08264118e-12  4.48434765e-12  5.94788425e-12  1.69295206e-12
 -3.67113872e-12 -5.28336721e-12 -1.8416701

In [82]:
dff4_th0 =pd.DataFrame(c_kte_th_f4 ,columns = ['F4/theta'])

dff4_al0 =pd.DataFrame(c_kte_al_f4 ,columns = ['F4/alpha'])

dff4_be0 =pd.DataFrame(c_kte_be_f4 ,columns = ['F4/beta'])
dff4_ga0 =pd.DataFrame(c_kte_ga_f4 ,columns = ['F4/gamma'])
dff4_F4 = pd.concat([dff4_th0,dff4_al0,dff4_be0,dff4_ga0],axis =1,sort=False)

#dff4_F4['phase'] = 1
dff4_F4

,F4/theta,F4/alpha,F4/beta,F4/gamma
0,1.639006e-12,0.0,1.639006e-12,1.639006e-12
1,4.528214e-12,0.0,4.528214e-12,4.528214e-12
2,2.774796e-12,0.0,2.774796e-12,2.774796e-12
3,-1.870277e-12,0.0,-1.870277e-12,-1.870277e-12
4,-5.180533e-12,0.0,-5.180533e-12,-5.180533e-12
...,...,...,...,...
422,4.294506e-12,0.0,4.294506e-12,4.294506e-12
423,5.059781e-13,0.0,5.059781e-13,5.059781e-13
424,-3.166632e-12,0.0,-3.166632e-12,-3.166632e-12
425,-2.698184e-12,0.0,-2.698184e-12,-2.698184e-12


In [83]:
# extract the frequency bands using bandpass filter
Th_f8_0 = filter(F8_0,4., 8.)
Al_f8_0 =filter(F8_0,8., 13.)
Be_f8_0 = filter(F8_0,13., 30.)
Ga_f8_0 = filter(F8_0,30., 56.)
#get the data from the signal
theta_f8,times = Th_f8_0[:]
alpha_f8,times = Al_f8_0[:]
beta_f8,times = Be_f8_0 [:]
gamma_f8,times = Ga_f8_0[:]

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 8 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 8.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 9.00 Hz)
- Filter length: 211 samples (1.648 sec)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 13 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 13.

In [84]:
F8_th0= TKEO(theta_f8)
F8_al0= TKEO(alpha_f8)
F8_be0= TKEO(beta_f8)
F8_ga0= TKEO(gamma_f8)
#############################
f8_th0 = Frame(F8_th0)
f8_al0 = Frame(F8_al0)
f8_be0 = Frame(F8_be0)
f8_ga0 = Frame(F8_ga0)

3 frame size
number_of_frames= 427
[1.93476195e-24 9.58568344e-26 2.89192048e-24]
frames len 427
(427, 3)
3 frame size
number_of_frames= 427
[ 7.14546738e-13 -3.12227433e-13 -9.10282512e-13]
frames len 427
(427, 3)
3 frame size
number_of_frames= 427
[0. 0. 0.]
frames len 427
(427, 3)
3 frame size
number_of_frames= 427
[ 7.14546738e-13 -3.12227433e-13 -9.10282512e-13]
frames len 427
(427, 3)


In [85]:
c_kte_th_f8 = coffe(f_th0)
c_kte_al_f8 = coffe(f8_al0)
c_kte_be_f8 = coffe(f8_be0)
c_kte_ga_f8 = coffe(f8_ga0)

[7.8125000e-03 3.1250000e-02 5.4687500e-02 7.8125000e-02 1.0156250e-01
 1.2500000e-01 1.4843750e-01 1.7187500e-01 1.9531250e-01 2.1875000e-01
 2.4218750e-01 2.6562500e-01 2.8906250e-01 3.1250000e-01 3.3593750e-01
 3.5937500e-01 3.8281250e-01 4.0625000e-01 4.2968750e-01 4.5312500e-01
 4.7656250e-01 5.0000000e-01 5.2343750e-01 5.4687500e-01 5.7031250e-01
 5.9375000e-01 6.1718750e-01 6.4062500e-01 6.6406250e-01 6.8750000e-01
 7.1093750e-01 7.3437500e-01 7.5781250e-01 7.8125000e-01 8.0468750e-01
 8.2812500e-01 8.5156250e-01 8.7500000e-01 8.9843750e-01 9.2187500e-01
 9.4531250e-01 9.6875000e-01 9.9218750e-01 1.0156250e+00 1.0390625e+00
 1.0625000e+00 1.0859375e+00 1.1093750e+00 1.1328125e+00 1.1562500e+00
 1.1796875e+00 1.2031250e+00 1.2265625e+00 1.2500000e+00 1.2734375e+00
 1.2968750e+00 1.3203125e+00 1.3437500e+00 1.3671875e+00 1.3906250e+00
 1.4140625e+00 1.4375000e+00 1.4609375e+00 1.4843750e+00 1.5078125e+00
 1.5312500e+00 1.5546875e+00 1.5781250e+00 1.6015625e+00 1.6250000e+00
 1.648

 4.98851618e+00 5.00027231e+00 5.01202844e+00]
[ 2.98006579e-12  6.03948744e-12  1.65752465e-12 -1.30081239e-12
 -4.42474788e-12 -3.61619109e-12  2.76453815e-12  3.96379212e-12
  1.13410540e-12 -7.96618912e-13 -3.05876881e-12 -1.15884790e-12
  1.93930105e-12 -6.23982004e-13 -4.45276674e-13  3.42544885e-12
  8.19700791e-13 -3.09111517e-12 -1.93061868e-12 -3.01034386e-13
  8.69953574e-13  2.51969474e-12  2.85007817e-12 -4.08233863e-13
 -5.68374311e-12 -4.36761066e-12  4.14073296e-12  6.89438056e-12
  7.59299149e-13 -4.23696043e-12 -4.28186008e-12 -1.60307317e-12
  2.76080126e-12  5.93133855e-12  2.70755173e-12 -5.71673372e-12
 -7.41125785e-12  1.61450321e-12  7.70614772e-12  3.09185869e-12
 -3.77823592e-12 -5.02618252e-12 -1.38046716e-12  2.86731983e-12
  4.48467756e-12  2.02837208e-12 -3.97131157e-12 -6.34016829e-12
  7.15027999e-13  6.86199210e-12  2.53814455e-12 -3.34637970e-12
 -2.88338787e-12 -1.01688001e-12 -1.76960504e-13  2.43183932e-12
  3.78397036e-12 -2.38409828e-13 -4.2604697

 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

In [86]:
dff8_th0 =pd.DataFrame(c_kte_th_f8 ,columns = ['F8/theta'])

dff8_al0 =pd.DataFrame(c_kte_al_f8 ,columns = ['F8/alpha'])

dff8_be0 =pd.DataFrame(c_kte_be_f8 ,columns = ['F8/beta'])
dff8_ga0 =pd.DataFrame(c_kte_ga_f8 ,columns = ['F8/gamma'])
dff8_F8 = pd.concat([dff8_th0,dff8_al0,dff8_be0,dff8_ga0],axis =1,sort=False)

#dff8_F8['phase'] = 1                       
dff8_F8

,F8/theta,F8/alpha,F8/beta,F8/gamma
0,0.003919,1.490033e-12,0.0,1.490033e-12
1,0.015675,3.019744e-12,0.0,3.019744e-12
2,0.027431,8.287623e-13,0.0,8.287623e-13
3,0.039187,-6.504062e-13,0.0,-6.504062e-13
4,0.050943,-2.212374e-12,0.0,-2.212374e-12
...,...,...,...,...
422,4.965004,-5.662006e-13,0.0,-5.662006e-13
423,4.976760,-4.977086e-13,0.0,-4.977086e-13
424,4.988516,-2.477112e-13,0.0,-2.477112e-13
425,5.000272,8.173393e-13,0.0,8.173393e-13


In [87]:
# extract the frequency bands using bandpass filter
Th_p2_0 = filter(Fp2_0,4., 8.)
Al_p2_0 =filter(Fp2_0,8., 13.)
Be_p2_0 = filter(Fp2_0,13., 30.)
Ga_p2_0 = filter(Fp2_0,30., 56.)
#get the data from the signal
theta_p2,times = Th_p2_0[:]
alpha_p2,times = Al_p2_0[:]
beta_p2,times = Be_p2_0 [:]
gamma_p2,times = Ga_p2_0[:]

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 8 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 8.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 9.00 Hz)
- Filter length: 211 samples (1.648 sec)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 13 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 13.

In [88]:
Fp2_th0= TKEO(theta_p2)
Fp2_al0= TKEO(alpha_p2)
Fp2_be0= TKEO(beta_p2)
Fp2_ga0= TKEO(gamma_p2)
#############################
fp2_th0 = Frame(Fp2_th0)
fp2_al0 = Frame(Fp2_al0)
fp2_be0 = Frame(Fp2_be0)
fp2_ga0 = Frame(Fp2_ga0)
############################
c_kte_th_p2 = coffe(fp2_th0)
c_kte_al_p2 = coffe(fp2_al0)
c_kte_be_p2 = coffe(fp2_be0)
c_kte_ga_p2 = coffe(fp2_ga0)

3 frame size
number_of_frames= 427
[-4.29845642e-12 -2.65348261e-12 -9.23440576e-13]
frames len 427
(427, 3)
3 frame size
number_of_frames= 427
[-4.29845642e-12 -2.65348261e-12 -9.23440576e-13]
frames len 427
(427, 3)
3 frame size
number_of_frames= 427
[-4.29845642e-12 -2.65348261e-12 -9.23440576e-13]
frames len 427
(427, 3)
3 frame size
number_of_frames= 427
[-4.29845642e-12 -2.65348261e-12 -9.23440576e-13]
frames len 427
(427, 3)
[ 1.89657417e-13  1.57802232e-12  2.30383174e-12 -8.28455936e-13
 -3.27227550e-12 -1.17796433e-12  2.47551753e-12  2.68868344e-12
 -1.12377817e-12 -2.44715369e-12  2.89380431e-13  5.73103825e-13
  2.70120725e-14  1.84633466e-12 -1.13182549e-13 -3.63039007e-12
 -4.58855550e-13  4.03827456e-12  2.02705992e-12 -2.79470273e-12
 -3.42221414e-12  1.14667538e-13  2.61135313e-12  2.67039413e-12
  9.60105978e-13 -3.37009528e-12 -4.97003569e-12  8.26057153e-13
  5.90094594e-12  3.42753687e-12 -2.48597212e-12 -5.80087514e-12
 -2.73453234e-12  3.75613743e-12  5.56448687

 1.17401071e-12 7.29751447e-13 1.33017804e-12]
[ 9.48287086e-14  7.89011161e-13  1.15191587e-12 -4.14227968e-13
 -1.63613775e-12 -5.88982164e-13  1.23775876e-12  1.34434172e-12
 -5.61889085e-13 -1.22357685e-12  1.44690216e-13  2.86551913e-13
  1.35060362e-14  9.23167328e-13 -5.65912743e-14 -1.81519504e-12
 -2.29427775e-13  2.01913728e-12  1.01352996e-12 -1.39735137e-12
 -1.71110707e-12  5.73337691e-14  1.30567656e-12  1.33519707e-12
  4.80052989e-13 -1.68504764e-12 -2.48501785e-12  4.13028576e-13
  2.95047297e-12  1.71376844e-12 -1.24298606e-12 -2.90043757e-12
 -1.36726617e-12  1.87806872e-12  2.78224343e-12  7.15871516e-13
 -1.96280522e-12 -2.77780760e-12 -2.88943784e-13  2.65045377e-12
  2.17643968e-12 -6.41774583e-13 -2.36846338e-12 -1.38553485e-12
  9.53783091e-13  1.87738058e-12  8.51078305e-13 -8.28240818e-13
 -1.79387007e-12 -5.86253681e-13  1.44828255e-12  1.23060120e-12
 -3.46274233e-13 -9.34805342e-13 -6.19099973e-13  2.09992217e-13
  9.93016410e-13  4.84285529e-13 -9.0597634

[ 1.89657417e-13  1.57802232e-12  2.30383174e-12 -8.28455936e-13
 -3.27227550e-12 -1.17796433e-12  2.47551753e-12  2.68868344e-12
 -1.12377817e-12 -2.44715369e-12  2.89380431e-13  5.73103825e-13
  2.70120725e-14  1.84633466e-12 -1.13182549e-13 -3.63039007e-12
 -4.58855550e-13  4.03827456e-12  2.02705992e-12 -2.79470273e-12
 -3.42221414e-12  1.14667538e-13  2.61135313e-12  2.67039413e-12
  9.60105978e-13 -3.37009528e-12 -4.97003569e-12  8.26057153e-13
  5.90094594e-12  3.42753687e-12 -2.48597212e-12 -5.80087514e-12
 -2.73453234e-12  3.75613743e-12  5.56448687e-12  1.43174303e-12
 -3.92561043e-12 -5.55561521e-12 -5.77887569e-13  5.30090753e-12
  4.35287935e-12 -1.28354917e-12 -4.73692675e-12 -2.77106970e-12
  1.90756618e-12  3.75476116e-12  1.70215661e-12 -1.65648164e-12
 -3.58774013e-12 -1.17250736e-12  2.89656511e-12  2.46120239e-12
 -6.92548465e-13 -1.86961068e-12 -1.23819995e-12  4.19984433e-13
  1.98603282e-12  9.68571057e-13 -1.81195269e-12 -1.78801453e-12
  1.24512096e-12  1.78335

In [89]:
dfp2_th0 =pd.DataFrame(c_kte_th_p2 ,columns = ['Fp2/theta'])

dfp2_al0 =pd.DataFrame(c_kte_al_p2 ,columns = ['Fp2/alpha'])

dfp2_be0 =pd.DataFrame(c_kte_be_p2 ,columns = ['Fp2/beta'])
dfp2_ga0 =pd.DataFrame(c_kte_ga_p2 ,columns = ['Fp2/gamma'])
dfp2_Fp2 = pd.concat([dfp2_th0,dfp2_al0,dfp2_be0,dfp2_ga0],axis =1,sort=False)

#dfp2_Fp2['phase'] = 1                       
dfp2_Fp2

,Fp2/theta,Fp2/alpha,Fp2/beta,Fp2/gamma
0,9.482871e-14,9.482871e-14,9.482871e-14,9.482871e-14
1,7.890112e-13,7.890112e-13,7.890112e-13,7.890112e-13
2,1.151916e-12,1.151916e-12,1.151916e-12,1.151916e-12
3,-4.142280e-13,-4.142280e-13,-4.142280e-13,-4.142280e-13
4,-1.636138e-12,-1.636138e-12,-1.636138e-12,-1.636138e-12
...,...,...,...,...
422,-3.200280e-12,-3.200280e-12,-3.200280e-12,-3.200280e-12
423,-7.116008e-13,-7.116008e-13,-7.116008e-13,-7.116008e-13
424,2.485331e-12,2.485331e-12,2.485331e-12,2.485331e-12
425,2.818211e-12,2.818211e-12,2.818211e-12,2.818211e-12


In [90]:
result = pd.concat([df0_Fp1,df7_F7,df3_F3,dfc1_FC1,
                    dfc2_FC2,dff4_F4,dff8_F8,dfp2_Fp2 ],axis =1,sort=False)
result['phase'] = 1
result.set_index('phase',inplace=True)
result
#save the data to a csv_file.
#result.to_csv('C:/Users/user/Desktop/feature/03062019 3/phase 1.csv'')

,FP1/theta,FP1/alpha,FP1/beta,FP1/gamma,F7/theta,F7/alpha,F7/beta,F7/gamma,F3/theta,F3/alpha,...,F4/beta,F4/gamma,F8/theta,F8/alpha,F8/beta,F8/gamma,Fp2/theta,Fp2/alpha,Fp2/beta,Fp2/gamma
phase,,,,,,,,,,,,,,,,,,,,,
1,0.003919,-1.098845e-12,-1.098845e-12,-1.098845e-12,1.351992e-12,1.351992e-12,1.351992e-12,1.351992e-12,-1.820290e-12,-1.820290e-12,...,1.639006e-12,1.639006e-12,0.003919,1.490033e-12,0.0,1.490033e-12,9.482871e-14,9.482871e-14,9.482871e-14,9.482871e-14
1,0.015675,-2.566065e-12,-2.566065e-12,-2.566065e-12,3.388368e-12,3.388368e-12,3.388368e-12,3.388368e-12,-5.891843e-12,-5.891843e-12,...,4.528214e-12,4.528214e-12,0.015675,3.019744e-12,0.0,3.019744e-12,7.890112e-13,7.890112e-13,7.890112e-13,7.890112e-13
1,0.027431,-8.364013e-13,-8.364013e-13,-8.364013e-13,1.652515e-12,1.652515e-12,1.652515e-12,1.652515e-12,-5.423363e-12,-5.423363e-12,...,2.774796e-12,2.774796e-12,0.027431,8.287623e-13,0.0,8.287623e-13,1.151916e-12,1.151916e-12,1.151916e-12,1.151916e-12
1,0.039187,1.231119e-12,1.231119e-12,1.231119e-12,-1.064408e-12,-1.064408e-12,-1.064408e-12,-1.064408e-12,1.364251e-12,1.364251e-12,...,-1.870277e-12,-1.870277e-12,0.039187,-6.504062e-13,0.0,-6.504062e-13,-4.142280e-13,-4.142280e-13,-4.142280e-13,-4.142280e-13
1,0.050943,2.029263e-12,2.029263e-12,2.029263e-12,-2.628500e-12,-2.628500e-12,-2.628500e-12,-2.628500e-12,7.825523e-12,7.825523e-12,...,-5.180533e-12,-5.180533e-12,0.050943,-2.212374e-12,0.0,-2.212374e-12,-1.636138e-12,-1.636138e-12,-1.636138e-12,-1.636138e-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,4.965004,1.757150e-12,1.757150e-12,1.757150e-12,-4.426104e-13,-4.426104e-13,-4.426104e-13,-4.426104e-13,1.681998e-12,1.681998e-12,...,4.294506e-12,4.294506e-12,4.965004,-5.662006e-13,0.0,-5.662006e-13,-3.200280e-12,-3.200280e-12,-3.200280e-12,-3.200280e-12
1,4.976760,7.371900e-13,7.371900e-13,7.371900e-13,-1.434249e-12,-1.434249e-12,-1.434249e-12,-1.434249e-12,1.618420e-12,1.618420e-12,...,5.059781e-13,5.059781e-13,4.976760,-4.977086e-13,0.0,-4.977086e-13,-7.116008e-13,-7.116008e-13,-7.116008e-13,-7.116008e-13
1,4.988516,-8.415997e-13,-8.415997e-13,-8.415997e-13,-4.500093e-13,-4.500093e-13,-4.500093e-13,-4.500093e-13,-1.271950e-12,-1.271950e-12,...,-3.166632e-12,-3.166632e-12,4.988516,-2.477112e-13,0.0,-2.477112e-13,2.485331e-12,2.485331e-12,2.485331e-12,2.485331e-12


In [91]:
result.to_csv('C:/Users/user/Desktop/feature/06062019/phase 1.csv')